# Model for recomendations solutions on drainage.

1. Postarać się dodać tagi z klasami zlewni

## 1. Import libraries

In [102]:
import swmmio
import pyswmm
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental import preprocessing


# desired_width = 500
# pd.set_option("display.width", desired_width)
# np.set_printoptions(linewidth=desired_width)
# pd.set_option("display.max_columns", 30)
import pprint
pd.set_option('display.max_columns', None)
# Ustawienie maksymalnej ilości wyświetlanych wierszy
pd.set_option('display.max_rows', None)


## 2. Recomendation classes

In [103]:
classes = pd.DataFrame(
    data={
        "classes": [
            "pump",
            "tank",
            "seepage_boxes",
            "diameter_increase",
            "diameter_reduction",
            "slope_increase",
            "slope_reduction",
            "depth_increase",
            "depth_reduction",
            "valid",
        ]
    }
)
classes

,classes
0,pump
1,tank
2,seepage_boxes
3,diameter_increase
4,diameter_reduction
5,slope_increase
6,slope_reduction
7,depth_increase
8,depth_reduction
9,valid


In [104]:
import sys
print(sys.path)

['c:\\Users\\Dell\\Documents\\Git\\stormwater-analysis\\stormwater_analysis\\data\\recomendations', 'C:\\Users\\Dell\\AppData\\Local\\Programs\\Python\\Python310\\python310.zip', 'C:\\Users\\Dell\\AppData\\Local\\Programs\\Python\\Python310\\DLLs', 'C:\\Users\\Dell\\AppData\\Local\\Programs\\Python\\Python310\\lib', 'C:\\Users\\Dell\\AppData\\Local\\Programs\\Python\\Python310', 'c:\\Users\\Dell\\Documents\\Git\\stormwater-analysis\\venv', '', 'c:\\Users\\Dell\\Documents\\Git\\stormwater-analysis\\venv\\lib\\site-packages', 'c:\\Users\\Dell\\Documents\\Git\\stormwater-analysis\\venv\\lib\\site-packages\\win32', 'c:\\Users\\Dell\\Documents\\Git\\stormwater-analysis\\venv\\lib\\site-packages\\win32\\lib', 'c:\\Users\\Dell\\Documents\\Git\\stormwater-analysis\\venv\\lib\\site-packages\\Pythonwin']


## 3. Get data

In [105]:
import os
os.chdir('C:\\Users\\Dell\\Documents\\Git\\stormwater-analysis')

from stormwater_analysis.inp_manage.inp import SwmmModel

In [106]:
INP_FILE_01 = R"C:\Users\Dell\Documents\Git\stormwater-analysis\stormwater_analysis\data\recomendations\dataset\01_recomendation.inp"
recomendations_01 = []

from stormwater_analysis.inp_manage.inp import SwmmModel
from stormwater_analysis.data.feature_engineering import feature_engineering

model = swmmio.Model(INP_FILE_01, include_rpt=True)

with pyswmm.Simulation(model.inp.path) as sim:
    for _ in sim:
        pass

conduits_data, nodes_data, subcatchments_data = feature_engineering(model)
swmm_model = SwmmModel(model, conduits_data, nodes_data, subcatchments_data)


1/1 [==============================] - 0s 27ms/step


### 3.1. Drop unused columns

In [107]:
swmm_model.conduits_data.conduits.drop(["Type", "MaxDay", "Tag", "InletNode", "OutletNode", "MaxHr"], inplace=True, axis=1)
print(swmm_model.conduits_data.conduits)

      Length  Roughness  MaxFlow  Geom1   MaxQ  MaxV  MaxQPerc  MaxDPerc  InletNodeInvert  OutletNodeInvert  UpstreamInvert  DownstreamInvert  SlopeFtPerFt  Filling  ValMaxFill  ValMaxV  ValMinV  ValMaxSlope  ValMinSlope  InletMaxDepth  OutletMaxDepth  SlopePerMile  InletGroundCover  OutletGroundCover  ValDepth  ValCoverage
Name                                                                                                                                                                                                                                                                                                                                 
C3        50      0.010        0    0.9  1.058  1.81      1.06      1.00           146.61            146.52          146.61            146.52        0.0018     0.90           0        1        1            1            1           2.71            2.40           1.8            143.90             144.12         1            1
C4        50      0.01

### 3.2. Categories 

In [108]:
classes = pd.DataFrame(
    data={
        "classes": [
            "pump",
            "tank",
            "seepage_boxes",
            "diameter_increase",
            "diameter_reduction",
            "slope_increase",
            "slope_reduction",
            "depth_increase",
            "depth_reduction",
            "valid",
        ]
    }
)
classes

,classes
0,pump
1,tank
2,seepage_boxes
3,diameter_increase
4,diameter_reduction
5,slope_increase
6,slope_reduction
7,depth_increase
8,depth_reduction
9,valid


### 3.3. Classification conduits to categories

In [109]:
print(swmm_model.conduits_data.conduits)
target = ["diameter_increase", "valid", "valid", "tank",  "valid", "valid", "valid", "valid", "valid", "valid", "valid", "valid", "valid", "valid", "valid", "valid", "valid", "valid", "valid", "valid", "valid",]

      Length  Roughness  MaxFlow  Geom1   MaxQ  MaxV  MaxQPerc  MaxDPerc  InletNodeInvert  OutletNodeInvert  UpstreamInvert  DownstreamInvert  SlopeFtPerFt  Filling  ValMaxFill  ValMaxV  ValMinV  ValMaxSlope  ValMinSlope  InletMaxDepth  OutletMaxDepth  SlopePerMile  InletGroundCover  OutletGroundCover  ValDepth  ValCoverage
Name                                                                                                                                                                                                                                                                                                                                 
C3        50      0.010        0    0.9  1.058  1.81      1.06      1.00           146.61            146.52          146.61            146.52        0.0018     0.90           0        1        1            1            1           2.71            2.40           1.8            143.90             144.12         1            1
C4        50      0.01

### 3.4. Merge target to conduits dataframe

In [110]:
swmm_model.conduits_data.conduits["target"] = ["diameter_increase", "valid", "valid", "tank",  "valid", "valid", "valid", "valid", "valid", "valid", "valid", "valid", "valid", "valid", "valid", "valid", "valid", "valid", "valid", "valid", "valid",]

### 3.5. Reset index

In [111]:
swmm_model.conduits_data.conduits.reset_index(drop=True, inplace=True)

In [112]:
print(swmm_model.conduits_data.conduits)

    Length  Roughness  MaxFlow  Geom1   MaxQ  MaxV  MaxQPerc  MaxDPerc  InletNodeInvert  OutletNodeInvert  UpstreamInvert  DownstreamInvert  SlopeFtPerFt  Filling  ValMaxFill  ValMaxV  ValMinV  ValMaxSlope  ValMinSlope  InletMaxDepth  OutletMaxDepth  SlopePerMile  InletGroundCover  OutletGroundCover  ValDepth  ValCoverage             target
0       50      0.010        0    0.9  1.058  1.81      1.06      1.00           146.61            146.52          146.61            146.52        0.0018     0.90           0        1        1            1            1           2.71            2.40           1.8            143.90             144.12         1            1  diameter_increase
1       50      0.015        0    1.0  1.479  2.39      0.89      0.74           146.52            146.20          146.52            146.20        0.0064     0.74           1        1        1            1            1           2.40            2.08           6.4            144.12             144.12         1    

In [113]:
swmm_model.conduits_data.conduits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Length             21 non-null     int64  
 1   Roughness          21 non-null     float64
 2   MaxFlow            21 non-null     int64  
 3   Geom1              21 non-null     float64
 4   MaxQ               21 non-null     float64
 5   MaxV               21 non-null     float64
 6   MaxQPerc           21 non-null     float64
 7   MaxDPerc           21 non-null     float64
 8   InletNodeInvert    21 non-null     float64
 9   OutletNodeInvert   21 non-null     float64
 10  UpstreamInvert     21 non-null     float64
 11  DownstreamInvert   21 non-null     float64
 12  SlopeFtPerFt       21 non-null     float64
 13  Filling            21 non-null     float64
 14  ValMaxFill         21 non-null     int32  
 15  ValMaxV            21 non-null     int32  
 16  ValMinV            21 non-nu

## 4. Split data into features and target

In [117]:
X = swmm_model.conduits_data.conduits.drop('target', axis=1)
y = swmm_model.conduits_data.conduits['target']

X = X.astype('float32')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(16, 26) (16, 2)
(5, 26) (5, 2)


In [121]:
print(y)

0     diameter_increase
1                 valid
2                 valid
3                  tank
4                 valid
5                 valid
6                 valid
7                 valid
8                 valid
9                 valid
10                valid
11                valid
12                valid
13                valid
14                valid
15                valid
16                valid
17                valid
18                valid
19                valid
20                valid
Name: target, dtype: object


In [118]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16 entries, 5 to 6
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Length             16 non-null     float32
 1   Roughness          16 non-null     float32
 2   MaxFlow            16 non-null     float32
 3   Geom1              16 non-null     float32
 4   MaxQ               16 non-null     float32
 5   MaxV               16 non-null     float32
 6   MaxQPerc           16 non-null     float32
 7   MaxDPerc           16 non-null     float32
 8   InletNodeInvert    16 non-null     float32
 9   OutletNodeInvert   16 non-null     float32
 10  UpstreamInvert     16 non-null     float32
 11  DownstreamInvert   16 non-null     float32
 12  SlopeFtPerFt       16 non-null     float32
 13  Filling            16 non-null     float32
 14  ValMaxFill         16 non-null     float32
 15  ValMaxV            16 non-null     float32
 16  ValMinV            16 non-nul

## 5. Normalization

In [119]:
normalizer = preprocessing.Normalization()
normalizer.adapt(np.array(X_train))

## 6. Build the model

In [122]:
model = Sequential()
model.add(normalizer)
model.add(Dense(units=X_train.shape[1], activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.2))
model.add(Dense(18, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=y_train.shape[1], activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_3 (Normalizat  (None, 26)               53        
 ion)                                                            
                                                                 
 dense (Dense)               (None, 26)                702       
                                                                 
 dropout (Dropout)           (None, 26)                0         
                                                                 
 dense_1 (Dense)             (None, 18)                486       
                                                                 
 dropout_1 (Dropout)         (None, 18)                0         
                                                                 
 dense_2 (Dense)             (None, 2)                 38        
                                                        

In [124]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
history = model.fit(
    x=X_train,
    y=y_train,
    epochs=100,
    validation_data=(X_test, y_test),
    validation_split=0.2,
    verbose=1,
    batch_size=16,
    callbacks=[EarlyStopping(monitor='val_loss', patience=10)]
)

Epoch 1/100
1/1 [==============================] - 1s 693ms/step - loss: 0.5633 - accuracy: 0.6875 - val_loss: 25330.6211 - val_accuracy: 0.4000
Epoch 2/100
1/1 [==============================] - 0s 47ms/step - loss: 0.5289 - accuracy: 0.8125 - val_loss: 28062.4883 - val_accuracy: 0.4000
Epoch 3/100
1/1 [==============================] - 0s 42ms/step - loss: 0.5923 - accuracy: 0.5625 - val_loss: 30770.0898 - val_accuracy: 0.4000
Epoch 4/100
1/1 [==============================] - 0s 51ms/step - loss: 0.5721 - accuracy: 0.6875 - val_loss: 33464.4609 - val_accuracy: 0.4000
Epoch 5/100
1/1 [==============================] - 0s 43ms/step - loss: 0.4964 - accuracy: 0.8125 - val_loss: 36112.6914 - val_accuracy: 0.4000
Epoch 6/100
1/1 [==============================] - 0s 46ms/step - loss: 0.5633 - accuracy: 0.6250 - val_loss: 38757.9922 - val_accuracy: 0.4000
Epoch 7/100
1/1 [==============================] - 0s 79ms/step - loss: 0.5566 - accuracy: 0.8125 - val_loss: 41422.1016 - val_accuracy

## 7. Evaluate model

In [128]:
# Evaluate ANN model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test set accuracy: {accuracy:.3f}")

1/1 [==============================] - 0s 39ms/step - loss: 51043.4062 - accuracy: 0.4000
Test set accuracy: 0.400


In [129]:
# model.predict(X_test)
y_pred = model.predict(X_test)

1/1 [==============================] - 0s 32ms/step


In [130]:
metrics = pd.DataFrame(history.history)
metrics['epoch'] = history.epoch
metrics

,loss,accuracy,val_loss,val_accuracy,epoch
0,0.563314,0.6875,25330.621094,0.4,0
1,0.528941,0.8125,28062.488281,0.4,1
2,0.592334,0.5625,30770.089844,0.4,2
3,0.572117,0.6875,33464.460938,0.4,3
4,0.496400,0.8125,36112.691406,0.4,4
5,0.563251,0.6250,38757.992188,0.4,5
6,0.556574,0.8125,41422.101562,0.4,6
7,0.505957,0.8750,44041.386719,0.4,7
8,0.488911,0.8125,46468.867188,0.4,8
9,0.490384,0.8125,48767.660156,0.4,9


In [131]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['accuracy'], name='accuracy'), row=1, col=1)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['loss'], name='loss'), row=1, col=2)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['val_accuracy'], name='val_accuracy'), row=1, col=1)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['val_loss'], name='val_loss'), row=1, col=2)

fig.update_xaxes(title_text='epochs')
fig.update_yaxes(title_text='accuracy', row=1, col=1)
fig.update_yaxes(title_text='loss', row=1, col=2)
fig.update_layout(width=1000, title='Accuracy and Loss')
fig.show()

In [132]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(test_acc)

0.4000000059604645


## 8. Predictions

In [133]:
predictions = model.predict(X_test)
predictions

1/1 [==============================] - 0s 50ms/step


array([[0.        , 1.        ],
       [0.24563791, 0.75436205],
       [0.71786714, 0.28213286],
       [0.        , 1.        ],
       [0.5262845 , 0.47371545]], dtype=float32)

In [134]:
predictions_df = pd.DataFrame(predictions)
predictions_df

,0,1
0,0.000000,1.000000
1,0.245638,0.754362
2,0.717867,0.282133
3,0.000000,1.000000
4,0.526285,0.473715


In [135]:
predictions_cls = predictions.argmax(axis=-1)
predictions_cls

array([1, 1, 0, 1, 0], dtype=int64)